### google image downloadを使って、１タレントあたり300～400枚の画像をダウンロード
#### requirement
- google_images_download(https://github.com/hardikvasa/google-images-download)
- chromeDriver(https://sites.google.com/a/chromium.org/chromedriver/downloads)

In [2]:
from google_images_download import google_images_download
import urllib
import pandas as pd
import bs4
import json
import os

In [3]:
response = google_images_download.googleimagesdownload()

In [4]:
chromedriver_path = "C:/Program Files/chromedriver_win32/chromedriver.exe"

In [5]:
def get_full_image_link(query, chromedriver_path, start=0):
    screen_width = 1920
    screen_height = 1080
    params = {
        "q": query,
        "sa": "X",
        "biw": screen_width,
        "bih": screen_height,
        "tbm": "isch",
        "start": start,
        #"ei": "" - This seems like a unique ID, you might want to use it to avoid getting banned. But you probably still are.
    }
    print(query)
    
    url="https://www.google.co.jp/search"
    
    URL = '{}?{}'.format(url, urllib.parse.urlencode(params))
    print(URL)
    
    source = response.download_extended_page(URL, chromedriver_path)
    
    bs = bs4.BeautifulSoup(source, 'html.parser')

    ActualImages=[]# contains the link for Large original images, type of  image
    
    cnt = 1
    for a in bs.findAll("div",{"class":"rg_meta"}):
        link , Type =json.loads(a.text)["ou"]  ,json.loads(a.text)["ity"]
        ActualImages.append((link,Type))
    
    return ActualImages

In [6]:
#タレント一覧の読み込み
cast_list = pd.read_csv("unique_cast_list.csv",encoding="utf-8-sig")

In [7]:
# image_links = get_full_image_link(cast_name.encode('utf-8'), chromedriver_path)
talent_name = "江口洋介"
talent_name = "松本幸四郎(市川染五郎)"
talent_name = "ロバート"

cast_list[cast_list.cast_name == talent_name]

,cast_name
407,ロバート


In [7]:
#キャスト名を用いて、google画像検索を行い、画像をダウンロード
for idx, row in cast_list.iterrows():    
    if idx < 1980:
        continue
        
    keyword = row.cast_name
    print(idx, keyword)
    
    x = 0
    _query = keyword
    
    image_links = get_full_image_link(_query.encode('utf-8'), chromedriver_path)
    dir_name = _query.replace("/", " ").replace(":", " ")

    save_directory =  './talent_image/' + dir_name
    print(save_directory)

    for i , (img , Type) in enumerate( image_links):
        try:
            Type = Type if len(Type) > 0 else 'jpg'
    #             print("Downloading image {} ({}), type is {}".format(i, img, Type))
            is_dir = os.path.isdir(save_directory)

            if not is_dir:
    #                 print("make directory:", save_directory)
                os.mkdir(save_directory)

            img_path = os.path.join(save_directory , "img_"+str(i)+"."+Type)

            if os.path.isfile(img_path):
                print(img_path, "already exist")
            else:
                raw_img = urllib.request.urlopen(img, timeout=10).read()
                f = open(img_path, 'wb')
                f.write(raw_img)
                f.close()
        except Exception as e:
            print ("could not load : "+img)
            print (e)

1980 黒木メイサ
b'\xe9\xbb\x92\xe6\x9c\xa8\xe3\x83\xa1\xe3\x82\xa4\xe3\x82\xb5'
https://www.google.co.jp/search?q=%E9%BB%92%E6%9C%A8%E3%83%A1%E3%82%A4%E3%82%B5&sa=X&biw=1920&tbm=isch&bih=1080&start=0
Getting you a lot of images. This may take a few moments...
Reached end of Page.
./talent_image/黒木メイサ
could not load : https://geitopi.com/wp-content/uploads/20140424010.jpg
HTTP Error 403: Forbidden
could not load : https://i0.wp.com/kisekish.com/wp-content/uploads/20161209_1.jpg?fit=350%2C450
HTTP Error 400: Bad Request
could not load : https://geitopi.com/wp-content/uploads/2016112601.jpg
HTTP Error 403: Forbidden
could not load : https://%E8%8A%B8%E8%83%BD%E4%BA%BA%E3%81%AE%E6%81%AF%E5%AD%90%E5%A8%98%E7%89%B9%E9%9B%86.com/wp-content/uploads/2017/11/171104-0012-300x300.jpg
'latin-1' codec can't encode characters in position 0-8: ordinal not in range(256)
could not load : https://img.news.goo.ne.jp/picture/crankin/m_crankin-5977203.jpg
HTTP Error 404: Not Found
could not load : https://www.ci